<a href="https://colab.research.google.com/github/thenromanov/FeedbackSentimentModel/blob/main/FeedbackSentimentModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U pip setuptools wheel
%pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python -m spacy download ru_core_news_lg

2023-04-09 09:01:47.148243: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 09:01:48.201848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=aea9d10ba8e819330c0d367feec029ef65ae67e04523cd49ed1e6ac6e0b1bb99
  Stored in directory: /root/.

In [3]:
#pragma dataset init bank_data --size 3Gb

# TODO: fill dataset here
# Dataset will be created in /home/jupyter/mnt/datasets/bank_data

import requests
from urllib.parse import urlencode
from io import BytesIO
from zipfile import ZipFile

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.com/d/LWS_pOE5pGAvOg'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
response = requests.get(download_url)

dist_path = '/home/jupyter/mnt/datasets/bank_data/'
zipfile = ZipFile(BytesIO(response.content))
zipfile.extractall(path=dist_path)

In [10]:
import pandas as pd
import numpy as np
import spacy
from sklearn.model_selection import train_test_split

In [11]:
nlp = spacy.load("ru_core_news_lg") # предобученная модель русского языка

In [126]:
import os

data_path = '/home/jupyter/mnt/datasets/bank_data/'

for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/jupyter/mnt/datasets/bank_data/banki.csv
/home/jupyter/mnt/datasets/bank_data/bank-sentiment.csv
/home/jupyter/mnt/datasets/bank_data/test_for_participants.csv
/home/jupyter/mnt/datasets/bank_data/train.csv
/home/jupyter/mnt/datasets/bank_data/new_test.csv
/home/jupyter/mnt/datasets/bank_data/banks-ethic-DB - NEW.docx
/home/jupyter/mnt/datasets/bank_data/sentiment.csv
/home/jupyter/mnt/datasets/bank_data/1sentencenewtest.csv
/home/jupyter/mnt/datasets/bank_data/bank-categories.csv


In [7]:
train_data_path = data_path + 'train.csv'
train_data = pd.read_csv(train_data_path)
train_data.describe()

,Unnamed: 0
count,19361.000000
mean,10755.525283
std,6217.076236
min,0.000000
25%,5368.000000
50%,10757.000000
75%,16152.000000
max,21512.000000


In [8]:
train_data['sentiment'].value_counts() # количества каждого из знаков

−    10192
+     6262
?     2907
Name: sentiment, dtype: int64

In [22]:
X = train_data['sentence']
sentiment_y = train_data['sentiment']

In [50]:
train_X, val_X, sentiment_train_y, sentiment_val_y = train_test_split(X, sentiment_y, random_state = 0, test_size=0.3) 
# разбиваем датаест на данные для обучения и валидации

In [15]:
from spacy.lang.ru.stop_words import STOP_WORDS

# получаем список стоп-слов
stopwords = list(STOP_WORDS) 
print(len(stopwords))

768


In [16]:
import string

punct = string.punctuation

In [17]:
def data_cleaner(sentence): # очистка доков от стоп-слов и леммаизация (замена слов на их начальные формы)
  doc = nlp(sentence)
  tokens = []
  for token in doc:
    if token.lemma_ != '-PRON-':
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  clean_tokens = []
  for token in tokens:
    if token not in punct and token not in stopwords:
      clean_tokens.append(token)
  return clean_tokens

In [75]:
data_cleaner('очень интересный')

['интересный']

In [76]:
data_cleaner('понравилось обслуживание')

['понравиться', 'обслуживание']

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

In [71]:
tfidf = TfidfVectorizer(tokenizer = data_cleaner)
train_X_fit = tfidf.fit_transform(train_X)
tfidf_svm_sentiment_steps = [('tfidf', tfidf), ('svm', LinearSVC())]

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
pipe_tfidf_svm_sentiment = Pipeline(tfidf_svm_sentiment_steps) #tfidf ветроризация, классификатор метод опорных векторов

In [31]:
pipe_tfidf_svm_sentiment.fit(train_X, sentiment_train_y)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function data_cleaner at 0x7fdbc8c439d0>)),
                ('svm', LinearSVC())])

In [32]:
tfidf_svm_sentiment_pred_y = pipe_tfidf_svm_sentiment.predict(val_X)

In [38]:
print(classification_report(sentiment_val_y, tfidf_svm_sentiment_pred_y))
print("\n")
print(confusion_matrix(sentiment_val_y, tfidf_svm_sentiment_pred_y))

              precision    recall  f1-score   support

           +       0.87      0.88      0.87      1884
           ?       0.66      0.54      0.59       902
           −       0.90      0.94      0.92      3023

    accuracy                           0.86      5809
   macro avg       0.81      0.79      0.80      5809
weighted avg       0.85      0.86      0.86      5809



[[1658  135   91]
 [ 196  487  219]
 [  56  114 2853]]


In [39]:
tfidf_svm_sentiment_pred_probas = OneHotEncoder().fit_transform(tfidf_svm_sentiment_pred_y.reshape(-1, 1)).toarray()
roc_auc_score(sentiment_val_y.to_numpy(), tfidf_svm_sentiment_pred_probas, multi_class='ovr')

0.8562499912542133

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
tfidf_forest_sentiment_steps = [('tfidf', tfidf),('forest', RandomForestClassifier(random_state=1))]
pipe_tfidf_forest_sentiment = Pipeline(tfidf_forest_sentiment_steps) # tfidf векторизация, классификатор случайный лес

In [37]:
pipe_tfidf_forest_sentiment.fit(train_X, sentiment_train_y)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function data_cleaner at 0x7fdbc8c439d0>)),
                ('forest', RandomForestClassifier(random_state=1))])

In [40]:
tfidf_forest_sentiment_pred_y = pipe_tfidf_forest_sentiment.predict(val_X)

In [41]:
print(classification_report(sentiment_val_y, tfidf_forest_sentiment_pred_y))
print("\n")
print(confusion_matrix(sentiment_val_y, tfidf_forest_sentiment_pred_y))

              precision    recall  f1-score   support

           +       0.87      0.88      0.87      1884
           ?       0.67      0.54      0.60       902
           −       0.89      0.94      0.92      3023

    accuracy                           0.86      5809
   macro avg       0.81      0.79      0.80      5809
weighted avg       0.85      0.86      0.85      5809



[[1649  125  110]
 [ 187  489  226]
 [  52  118 2853]]


In [43]:
from sklearn.base import BaseEstimator, TransformerMixin

In [44]:
class WordVectorTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, model="ru_core_news_lg"):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        nlp = spacy.load(self.model)
        return np.concatenate([nlp(' '.join(data_cleaner(doc))).vector.reshape(1, -1) for doc in X])

In [45]:
word2vec_svm_sentiment_steps = [('vect', WordVectorTransformer()), ('svm', LinearSVC())]

In [48]:
pipe_word2vec_svm_sentiment = Pipeline(word2vec_svm_sentiment_steps)

In [49]:
pipe_word2vec_svm_sentiment.fit(train_X, sentiment_train_y) # word2vec векторизация, классификатор метод опорных векторов

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('vect', WordVectorTransformer()), ('svm', LinearSVC())])

In [52]:
word2vec_svm_sentiment_pred_y = pipe_word2vec_svm_sentiment.predict(val_X)

In [53]:
print(classification_report(sentiment_val_y, word2vec_svm_sentiment_pred_y))
print("\n")
print(confusion_matrix(sentiment_val_y, word2vec_svm_sentiment_pred_y))

              precision    recall  f1-score   support

           +       0.76      0.81      0.79      1884
           ?       0.58      0.21      0.31       902
           −       0.80      0.91      0.85      3023

    accuracy                           0.77      5809
   macro avg       0.71      0.65      0.65      5809
weighted avg       0.75      0.77      0.75      5809



[[1530   67  287]
 [ 296  193  413]
 [ 188   70 2765]]


In [54]:
word2vec_svm_sentiment_pred_probas = OneHotEncoder().fit_transform(word2vec_svm_sentiment_pred_y.reshape(-1, 1)).toarray()
roc_auc_score(sentiment_val_y.to_numpy(), word2vec_svm_sentiment_pred_probas, multi_class='ovr')

0.7563729172207715

In [119]:
word2vec_forest_sentiment_steps = [('vect', WordVectorTransformer()), ('forest', RandomForestClassifier(random_state=1))]

In [120]:
pipe_word2vec_forest_sentiment = Pipeline(word2vec_forest_sentiment_steps)

In [121]:
pipe_word2vec_forest_sentiment.fit(train_X, sentiment_train_y) # word2vec векторизация, классификатор случайный лес

Pipeline(steps=[('vect', WordVectorTransformer()),
                ('forest', RandomForestClassifier(random_state=1))])

In [122]:
word2vec_forest_sentiment_pred_y = pipe_word2vec_forest_sentiment.predict(val_X)

In [123]:
print(classification_report(sentiment_val_y, word2vec_forest_sentiment_pred_y))
print("\n")
print(confusion_matrix(sentiment_val_y, word2vec_forest_sentiment_pred_y)) # классификационыые метрики итоговой выбранной модели на сентименте

              precision    recall  f1-score   support

           +       0.88      0.87      0.87      1884
           ?       0.67      0.53      0.59       902
           −       0.89      0.95      0.91      3023

    accuracy                           0.86      5809
   macro avg       0.81      0.78      0.79      5809
weighted avg       0.85      0.86      0.85      5809



[[1632  121  131]
 [ 184  481  237]
 [  44  120 2859]]


In [124]:
word2vec_forest_sentiment_pred_y_probas = pipe_word2vec_forest_sentiment.predict_proba(val_X)

In [125]:
roc_auc_score(sentiment_val_y.to_numpy(), word2vec_forest_sentiment_pred_y_probas, multi_class='ovr') # roc-auc метрика итоговой выбранной модели на сентименте

0.9155151484735047

In [127]:
categories = sorted(list(train_data['1category'].unique()))
categories.remove('?')
print(categories)

['Communication', 'Price', 'Quality', 'Safety']


In [128]:
train_data_2 = train_data[~train_data['1category'].isnull()]
train_data_2 = train_data[~train_data['1category'].isin(['?'])]
category_X = train_data_2['sentence']
category_y = train_data_2['1category']

In [129]:
category_train_X, category_val_X, category_train_y, category_val_y = train_test_split(category_X, category_y, random_state = 0, test_size=0.3)

In [130]:
word2vec_forest_category_steps = [('vect', WordVectorTransformer()), ('forest', RandomForestClassifier(random_state=1))]

In [131]:
pipe_word2vec_forest_category = Pipeline(word2vec_forest_category_steps)

In [132]:
pipe_word2vec_forest_category.fit(category_train_X, category_train_y)

Pipeline(steps=[('vect', WordVectorTransformer()),
                ('forest', RandomForestClassifier(random_state=1))])

In [133]:
word2vec_forest_category_pred_y = pipe_word2vec_forest_category.predict(category_val_X)

In [134]:
print(classification_report(category_val_y, word2vec_forest_category_pred_y))
print("\n")
print(confusion_matrix(category_val_y, word2vec_forest_category_pred_y))

               precision    recall  f1-score   support

Communication       0.73      0.73      0.73      2188
        Price       0.52      0.45      0.48       104
      Quality       0.63      0.63      0.63      1683
       Safety       0.44      0.46      0.45        56

     accuracy                           0.68      4031
    macro avg       0.58      0.57      0.57      4031
 weighted avg       0.68      0.68      0.68      4031



[[1607   11  548   22]
 [   9   47   47    1]
 [ 583   32 1058   10]
 [  15    0   15   26]]


In [135]:
word2vec_forest_category_pred_proba_y = pipe_word2vec_forest_category.predict_proba(category_val_X)

In [136]:
roc_auc_score(category_val_y.to_numpy(), word2vec_forest_category_pred_proba_y, multi_class='ovr')

0.7997948326766495

In [137]:
test_data_path = data_path + '1sentencenewtest.csv'
test_data = pd.read_csv(test_data_path)

In [139]:
test_X = test_data['0']

In [140]:
test_sentiment_y = pipe_word2vec_forest_sentiment.predict_proba(test_X)

In [141]:
print(test_sentiment_y)

[[0.28466667 0.21033333 0.505     ]
 [0.215      0.292      0.493     ]
 [0.36       0.24016667 0.39983333]
 ...
 [0.309      0.323      0.368     ]
 [0.21083333 0.07       0.71916667]
 [0.34266667 0.25483333 0.4025    ]]


In [142]:
test_category_y = pipe_word2vec_forest_category.predict_proba(test_X)

In [143]:
print(test_category_y)

[[0.3495     0.08333333 0.5405     0.02666667]
 [0.28466667 0.01333333 0.655      0.047     ]
 [0.58583333 0.04       0.33916667 0.035     ]
 ...
 [0.39561905 0.09       0.45771429 0.05666667]
 [0.50183333 0.02       0.40316667 0.075     ]
 [0.37566667 0.139      0.47033333 0.015     ]]
